# Lesson 1: Creating an advanced RAG Pipeline

In [16]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

In [18]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./GK_Cover_Letter_Avanade.pdf"]
).load_data()

In [19]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

1 

<class 'llama_index.schema.Document'>
Doc ID: 991da2c0-7af7-4acf-942f-4ecdd5630792
Text: Dear Hiring Manager, I am writing to express my interest in the
Data Scientist position at Avanade. With over 7 years of  experience
in data science, business intelligence, and software development, I am
confident that my skills align perfectly with the requirements of the
role. My background in end-to-end machine  learning, data analysis,
and c...


## Basic RAG pipeline

In [20]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [21]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

In [24]:
query_engine = index.as_query_engine()

In [26]:
response = query_engine.query(
    "Which projects has gautam worked in his experience?"
)
print(str(response))

Gautam has worked on projects involving data pipelines, anomaly detection, retrieval-augmented generation (RAG) models, prompt engineering, fine-tuning, understanding vector indexes in real-time applications, and A/B testing.


## Evaluation setup using TruLens

In [27]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?
which countries Gautam lived in ?
Which companies gautam worked in ?
Which all skills Gautam posses ?
What roles Gautam has worked as ?


In [28]:
# You can try your own question:
new_question = "What is the right AI job for Gautam?"
eval_questions.append(new_question)

In [29]:
print(eval_questions)

['What are the keys to building a career in AI?', 'How can teamwork contribute to success in AI?', 'What is the importance of networking in AI?', 'What are some good habits to develop for a successful career?', 'How can altruism be beneficial in building a career?', 'What is imposter syndrome and how does it relate to AI?', 'Who are some accomplished individuals who have experienced imposter syndrome?', 'What is the first step to becoming good at AI?', 'What are some common challenges in AI?', 'Is it normal to find parts of AI challenging?', 'which countries Gautam lived in ?', 'Which companies gautam worked in ?', 'Which all skills Gautam posses ?', 'What roles Gautam has worked as ?', 'What is the right AI job for Gautam?']


In [30]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

For the classroom, we've written some of the code in helper functions inside a utils.py file.  
- You can view the utils.py file in the file directory by clicking on the "Jupyter" logo at the top of the notebook.
- In later lessons, you'll get to work directly with the code that's currently wrapped inside these helper functions, to give you more options to customize your RAG pipeline.

In [31]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [32]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [33]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [36]:
records.tail(10)

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Context Relevance,Answer Relevance,Groundedness,Context Relevance_calls,Answer Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
5,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_3245b0470811b570a762c4915adaf52b,"""What is imposter syndrome and how does it rel...","""Imposter syndrome is a psychological pattern ...",-,"{""record_id"": ""record_hash_3245b0470811b570a76...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-10-24T04:52:11.389196"", ""...",2024-10-24T04:52:12.925519,0.0,1.0,0.00,[{'args': {'prompt': 'What is imposter syndrom...,[{'args': {'prompt': 'What is imposter syndrom...,"[{'args': {'source': 'Dear Hiring Manager, I a...",1,522,0.000831
6,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_495b754b813dd9e6b6365d9b8738a465,"""Who are some accomplished individuals who hav...","""Many accomplished individuals, such as Maya A...",-,"{""record_id"": ""record_hash_495b754b813dd9e6b63...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-10-24T04:52:13.029907"", ""...",2024-10-24T04:52:16.254478,0.0,1.0,0.00,[{'args': {'prompt': 'Who are some accomplishe...,[{'args': {'prompt': 'Who are some accomplishe...,"[{'args': {'source': 'Dear Hiring Manager, I a...",3,454,0.000696
7,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_4110edab85ff42e6207bbac89d57811b,"""What is the first step to becoming good at AI?""","""Developing a strong foundation in programming...",-,"{""record_id"": ""record_hash_4110edab85ff42e6207...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-10-24T04:52:16.357249"", ""...",2024-10-24T04:52:17.148729,0.0,1.0,0.50,[{'args': {'prompt': 'What is the first step t...,[{'args': {'prompt': 'What is the first step t...,"[{'args': {'source': 'Dear Hiring Manager, I a...",0,446,0.000680
8,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_5aefab981ab965db849fa658d29fdfb5,"""What are some common challenges in AI?""","""Some common challenges in AI include data qua...",-,"{""record_id"": ""record_hash_5aefab981ab965db849...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-10-24T04:52:17.249780"", ""...",2024-10-24T04:52:18.254795,0.0,1.0,0.18,[{'args': {'prompt': 'What are some common cha...,[{'args': {'prompt': 'What are some common cha...,"[{'args': {'source': 'Dear Hiring Manager, I a...",1,467,0.000723
9,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_fa06ec8cfd168ef7db1ac521927fd2f1,"""Is it normal to find parts of AI challenging?""","""Yes, it is normal to find parts of AI challen...",-,"{""record_id"": ""record_hash_fa06ec8cfd168ef7db1...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-10-24T04:52:18.361352"", ""...",2024-10-24T04:52:19.294723,0.0,1.0,NaN,[{'args': {'prompt': 'Is it normal to find par...,[{'args': {'prompt': 'Is it normal to find par...,NaN,0,459,0.000706
10,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_0a1edd3906633a0f77ce0cc123a52c09,"""which countries Gautam lived in ?""","""The context information does not provide any ...",-,"{""record_id"": ""record_hash_0a1edd3906633a0f77c...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-10-24T04:52:19.389982"", ""...",2024-10-24T04:52:20.108160,NaN,NaN,NaN,NaN,NaN,NaN,0,437,0.000664
11,Direct Query Engi

In [37]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-29-38-34p38560.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Advanced RAG pipeline

### 1. Sentence Window retrieval

In [38]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [39]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [40]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [41]:
window_response = sentence_window_engine.query(
    "Which project Gautam worked on before amazon web services?"
)
print(str(window_response))

Gautam worked on projects involving data pipelines and anomaly detection before Amazon Web Services.


In [42]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [43]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

What are the keys to building a career in AI?
Developing expertise in data science, machine learning, and cloud computing, along with proficiency in programming languages like Python and SQL, are key components to building a successful career in AI. Additionally, hands-on experience with tools and platforms such as Azure AI, AWS, Redshift, Azure ML Studio, and Databricks can further enhance one's capabilities in the field. Continuous learning, staying updated with the latest trends, and actively working on projects involving data analysis, modeling, and machine learning are essential for advancing a career in AI.
How can teamwork contribute to success in AI?
Collaboration and teamwork are essential in AI projects as they bring together diverse perspectives, expertise, and skills. By working in a team, individuals can leverage each other's strengths, share knowledge, and collectively solve complex problems more effectively. Teamwork fosters innovation, encourages creativity, and promote

In [46]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
Sentence Window Query Engine,0.339444,1.0,0.04,1.333333,0.000724


In [48]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-29-38-34p38560.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### 2. Auto-merging retrieval

In [52]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [53]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [54]:
auto_merging_response = automerging_query_engine.query(
    "How should Gautam improve his job prospects ?"
)
print(str(auto_merging_response))

> Merging 4 nodes into parent node.
> Parent node id: 9ff86d3d-775b-4e39-92d3-e1d995ef31df.
> Parent node text: Dear Hiring Manager,
I am writing to express my interest in the Data Scientist position at Avanad...

> Merging 1 nodes into parent node.
> Parent node id: 6a410070-4faa-4bb7-9c7f-e58370347a52.
> Parent node text: Dear Hiring Manager,
I am writing to express my interest in the Data Scientist position at Avanad...

Gautam can improve his job prospects by highlighting specific achievements and quantifiable results from his previous work experiences. Additionally, he should tailor his resume and cover letter to align closely with the requirements of the positions he is applying for at different companies. Networking with professionals in the industry and staying updated on the latest trends and technologies in data science can also enhance his job prospects.


In [55]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [56]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

> Merging 4 nodes into parent node.
> Parent node id: 9ff86d3d-775b-4e39-92d3-e1d995ef31df.
> Parent node text: Dear Hiring Manager,
I am writing to express my interest in the Data Scientist position at Avanad...

> Merging 1 nodes into parent node.
> Parent node id: 6a410070-4faa-4bb7-9c7f-e58370347a52.
> Parent node text: Dear Hiring Manager,
I am writing to express my interest in the Data Scientist position at Avanad...

What are the keys to building a career in AI?
Developing expertise in data science, business intelligence, and software development is essential for building a successful career in AI. Proficiency in end-to-end machine learning, data analysis, and cloud computing, along with strong skills in Python and SQL, are key components. Hands-on experience with building and deploying machine learning models, working with data pipelines, and understanding statistical concepts like hypothesis testing are also crucial for advancing in the field of AI. Additionally, staying updat

Is it normal to find parts of AI challenging?
Yes, it is normal to find parts of AI challenging, especially when working on complex tasks such as building and deploying machine learning models, optimizing data storage, and working with real-time applications. Understanding statistical concepts, fine-tuning models, and handling large datasets can also present challenges in the field of AI.
> Merging 4 nodes into parent node.
> Parent node id: 9ff86d3d-775b-4e39-92d3-e1d995ef31df.
> Parent node text: Dear Hiring Manager,
I am writing to express my interest in the Data Scientist position at Avanad...

> Merging 1 nodes into parent node.
> Parent node id: 6a410070-4faa-4bb7-9c7f-e58370347a52.
> Parent node text: Dear Hiring Manager,
I am writing to express my interest in the Data Scientist position at Avanad...

which countries Gautam lived in ?
The information provided does not mention any specific countries where Gautam Kumar has lived.
> Merging 4 nodes into parent node.
> Parent node i

In [57]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
Automerging Query Engine,0.136111,1.0,0.0,0.933333,0.000788


In [58]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-29-38-34p38560.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>